In [25]:
import MeCab
m = MeCab.Tagger()
a = m.parse("오늘은 좋은날, 행복한 삶을 누리자.")
print(a)
print(m.parse("오늘은 좋은날, 11행복한 삶을 누리자.").split('\n'))
a = list(map(lambda morphs : morphs.split('\t')[0], m.parse("오늘은 좋은날, 11행복한 삶을 누리자.").split('\n')))[:-2]
# a = list(map(lambda morphs : morphs[0], m.parse("오늘은 좋은날, 11행복한 삶을 누리자.").split('\n')))
print(a)
# print(m.('화학 이외의 것'))


오늘	NNG,*,T,오늘,*,*,*,*
은	JX,*,T,은,*,*,*,*
좋	VA,*,T,좋,*,*,*,*
은	ETM,*,T,은,*,*,*,*
날	NNG,*,T,날,*,*,*,*
,	SC,*,*,*,*,*,*,*
행복	NNG,정적사태,T,행복,*,*,*,*
한	XSA+ETM,*,T,한,Inflect,XSA,ETM,하/XSA/*+ᆫ/ETM/*
삶	NNG,행위,T,삶,*,*,*,*
을	JKO,*,T,을,*,*,*,*
누리	VV,*,F,누리,*,*,*,*
자	EF,*,F,자,*,*,*,*
.	SF,*,*,*,*,*,*,*
EOS

['오늘\tNNG,*,T,오늘,*,*,*,*', '은\tJX,*,T,은,*,*,*,*', '좋\tVA,*,T,좋,*,*,*,*', '은\tETM,*,T,은,*,*,*,*', '날\tNNG,*,T,날,*,*,*,*', ',\tSC,*,*,*,*,*,*,*', '11\tSN,*,*,*,*,*,*,*', '행복\tNNG,정적사태,T,행복,*,*,*,*', '한\tXSA+ETM,*,T,한,Inflect,XSA,ETM,하/XSA/*+ᆫ/ETM/*', '삶\tNNG,행위,T,삶,*,*,*,*', '을\tJKO,*,T,을,*,*,*,*', '누리\tVV,*,F,누리,*,*,*,*', '자\tEF,*,F,자,*,*,*,*', '.\tSF,*,*,*,*,*,*,*', 'EOS', '']
['오늘', '은', '좋', '은', '날', ',', '11', '행복', '한', '삶', '을', '누리', '자', '.']


In [18]:
import MeCab
mecab = MeCab.()

mecab.morphs('영등포구청역에 있는 맛집 좀 알려주세요.')

ModuleNotFoundError: No module named 'mecab'

In [16]:
import os
import json
from tqdm import tqdm
import re
import MeCab
import easydict
from prepro.data_builder import format_to_bert
def run(path):
    for path2 in ['train','valid']:
        DATAPATH = path+path2
        filenames = [x for x in os.listdir (DATAPATH) if x.endswith('json')]
        list_dic = []

        for file in filenames:
            filelocation = os.path.join(DATAPATH, file)

            with open(filelocation, 'r',encoding='utf-8') as json_file:
                data = json.load(json_file)['documents']

                for x in tqdm (range(len(data))):
                    text = data[x]['text']
                    text = str(text).replace('"', "'")

                    extractive = data[x]['extractive']
                    for index, value in enumerate(extractive):
                        if value == None:
                            extractive[index] = 0

                    p = re.compile('(?<=sentence\'\: \')(.*?)(?=\'highlight_indices)')
                    texts = p.findall(text)

                    sentences = []
                    for t in texts:
                        sentence = t[:-3]
                        sentences.append(sentence)

                    mydict = {}
                    mydict['text'] = sentences
                    mydict['extractive'] = extractive
                    list_dic.append(mydict)
        
        with open(f"{path+path2}.json", 'w') as fh:
            json.dump(list_dic, fh)

        def list_chunk(lst, n):
            return [lst[i:i+n] for i in range(0, len(lst), n)]
        if path2 == 'train':
            with open(f"{path+path2}.json", 'r') as fh:
                data = json.load(fh)
            data_chunked = list_chunk(data, 32507) ## 전체 데이터를 10개로 분할
            for i, d in enumerate(data_chunked):
                with open(f"{path+path2}.{i}.json".format(i), 'w') as fh:
                    json.dump(d, fh)
        mecab = MeCab.Tagger()
        DATAPATH = path
        filenames = [x for x in os.listdir (DATAPATH) if path2 in x and x.endswith('json')]
        print(filenames)
        trainfiles = []
        for f in filenames[1:-1]:
            trainfiles.append(f[:-5])
        print(trainfiles)
        for set_name in trainfiles:
            print("Processing ... ", set_name)

            with open("{}/{}.json".format(path,set_name), 'r',encoding='utf-8') as f:
                data = json.load(f)

                list_dic = []
                for x in tqdm(range(len(data))):
                    text = data[x]['text']
                    extractive = data[x]['extractive']

                    sentences = []
                    for sentence in text:
                        sentence_morph = ' '.join(list(map(lambda morphs : morphs.split('\t')[0], m.parse(sentence).split('\n')))[:-2])
                        sentences.append(sentence_morph)

                    extractives = []
                    for e in extractive:
                        extractives.append(sentences[e])

                    src = [i.split(' ') for i in sentences]
                    tgt = [i.split(' ') for i in extractives]

                    mydict = {}
                    mydict['src'] = src
                    mydict['tgt'] = tgt
                    list_dic.append(mydict)

                jsonfilelocation = path+'../json_data/' + path2
                os.makedirs(jsonfilelocation, exist_ok=True)

                temp = []
                DATA_PER_FILE = 50

                for i,a in enumerate(tqdm(list_dic)):
                    if (i+1)%DATA_PER_FILE!=0:
                        temp.append(a)
                    else:
                        temp.append(a)
                        filename = 'korean.'+ set_name + '.' + str(i//DATA_PER_FILE)+'.json'
                        with open(os.path.join(jsonfilelocation, filename), "w", encoding='utf-8') as json_file:
                            json.dump(temp, json_file, ensure_ascii=False)
                        temp = []

                    #마지막에 남은 데이터 있으면 추가로 append
                    if len(temp) != 0:
                        filename = 'korean.'+ set_name + '.' + str(i//DATA_PER_FILE + 1)+'.json'
                        with open(os.path.join(jsonfilelocation, filename), "w", encoding='utf-8') as json_file:
                            json.dump(temp, json_file, ensure_ascii=False)
        set_name = path2

        bertfilelocation = path+'../bert_data/'
        os.makedirs(bertfilelocation, exist_ok=True)
        print(bertfilelocation, path+'../json_data/')
        args = easydict.EasyDict({
        "dataset": [set_name], 
        "raw_path": path+'../json_data/',
        "save_path": bertfilelocation,
        "n_cpus":16,
        "oracle_mode": "greedy",
        "min_src_ntokens": 5,
        "max_src_ntokens": 200,
        "min_nsents": 3,
        "max_nsents": 100,
        })
        format_to_bert(args)
if __name__ == '__main__':
    run('../raw_data/')

../raw_data/../bert_data/ ../raw_data/../json_data/
../raw_data/../bert_data/ train\korean.train.1.0.json
../raw_data/../json_data/train\korean.train.1.0.json {'dataset': ['train'], 'raw_path': '../raw_data/../json_data/', 'save_path': '../raw_data/../bert_data/', 'n_cpus': 16, 'oracle_mode': 'greedy', 'min_src_ntokens': 5, 'max_src_ntokens': 200, 'min_nsents': 3, 'max_nsents': 100} ../raw_data/../bert_data/train\korean.train.1.0.bert.pt
../raw_data/../bert_data/ ../raw_data/../json_data/


In [5]:
!pip install pyrouge --upgrade

!pip install https://github.com/bheinzerling/pyrouge/archive/master.zip

!pip install pyrouge

!pip show pyrouge

!git clone https://github.com/andersjo/pyrouge.git

     ---------------------------------------- 60.5/60.5 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for pyrouge: filename=pyrouge-0.1.3-py3-none-any.whl size=191648 sha256=4a2ef18524525589250c7a8d1e565a13156a5fabcf1cf0fd9c8fdc481dbc2d8f
  Stored in directory: c:\users\answl\appdata\local\pip\cache\wheels\00\85\fd\ccd28e53c9f6a691e6ea96050a0cad95f9a4a6361269d765ca
Successfully built pyrouge
     | 202.3 kB 336.9 kB/s 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
Name: pyrouge
Version: 0.1.3
Summary: A Python wrapper for the ROUGE summarization evaluation package.
Home-page: https://github.com/noutenki/pyrouge
Author: Benjamin Heinzerling, Anders Johannsen
Author-email: benjamin.heinzerling@h-its.org
License: LICENSE.txt
Location: c:\users\answl\anaconda3\envs\nlp\lib\site-packages
Requires: 
Required-by: 


Cloning into 'pyrouge'...


In [3]:
from pyrouge import Rouge155

!pyrouge_set_rouge_path '/content/pyrouge/tools/ROUGE-1.5.5'

ModuleNotFoundError: No module named 'konlpy'

In [5]:
!python SRC/train.py

gpu_rank 0


[2022-11-05 13:04:59,763 INFO] Device ID -1
[2022-11-05 13:04:59,763 INFO] Device cpu

Downloading:   0%|          | 0.00/425 [00:00<?, ?B/s]
Downloading: 100%|██████████| 425/425 [00:00<00:00, 425kB/s]
c:\Users\answl\anaconda3\envs\nlp\lib\site-packages\huggingface_hub\file_download.py:123: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\answl\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-start